<div class="alert alert-success" role="quote" style="text-align:center;font-size:120%;;">
  In the name of Allah, the most Merciful and the most Beneficent!</div>

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
import xgboost as xgb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Prices

In [ ]:
prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")
secondary = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/secondary_stock_prices.csv")


In [ ]:
prices.drop(["RowId", "ExpectedDividend", "SupervisionFlag","AdjustmentFactor"], axis=1, inplace=True)

In [ ]:
secondary.drop(["RowId", "ExpectedDividend", "SupervisionFlag","AdjustmentFactor"], axis=1, inplace=True)

In [ ]:
prices.head()

# Merging 

In [ ]:
df = pd.merge(prices, secondary, how="outer", sort=True)

# Hope that there is no NaN :)

In [ ]:
import missingno

missingno.matrix(df);

In [ ]:
df = df.dropna()

In [ ]:
missingno.matrix(df);

# Go to Add Indicators

In [ ]:
def EVM(data, ndays): 
    dm = ((data['High'] + data['Low'])/2) - ((data['High'].shift(1) + data['Low'].shift(1))/2)
    br = (data['Volume'] / 100000000) / ((data['High'] - data['Low']))
    EVM = dm / br 
    EVM_MA = pd.Series(EVM.rolling(ndays).mean(), name = 'EVM_'+str(ndays)) 
    data = data.join(EVM_MA) 
    return data


# Exponentially-weighted Moving Average 
def EWMA(data, ndays): 
    EMA = pd.Series(data['Close'].ewm(span = ndays, min_periods = ndays - 1).mean(), 
                    name = 'EWMA_' + str(ndays)) 
    data = data.join(EMA) 
    return data


# Rate of Change (ROC)
def ROC(data,n):
    N = data['Close'].diff(n)
    D = data['Close'].shift(n)
    ROC = pd.Series(N/D,name='Rate_of_Change_'+str(n))
    data = data.join(ROC)
    return data 


def ADX(data: pd.DataFrame, period: int):
    """
    Computes the ADX indicator.
    """
    
    df = data.copy()
    alpha = 1/period

    # TR
    df['H-L'] = df['High'] - df['Low']
    df['H-C'] = np.abs(df['High'] - df['Close'].shift(1))
    df['L-C'] = np.abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['H-L', 'H-C', 'L-C']].max(axis=1)
    del df['H-L'], df['H-C'], df['L-C']

    # ATR
    df['ATR'] = df['TR'].ewm(alpha=alpha, adjust=False).mean()

    # +-DX
    df['H-pH'] = df['High'] - df['High'].shift(1)
    df['pL-L'] = df['Low'].shift(1) - df['Low']
    df['+DX'] = np.where(
        (df['H-pH'] > df['pL-L']) & (df['H-pH']>0),
        df['H-pH'],
        0.0
    )
    df['-DX'] = np.where(
        (df['H-pH'] < df['pL-L']) & (df['pL-L']>0),
        df['pL-L'],
        0.0
    )
    del df['H-pH'], df['pL-L']

    # +- DMI
    df['S+DM'] = df['+DX'].ewm(alpha=alpha, adjust=False).mean()
    df['S-DM'] = df['-DX'].ewm(alpha=alpha, adjust=False).mean()
    df['+DMI'] = (df['S+DM']/df['ATR'])*100
    df['-DMI'] = (df['S-DM']/df['ATR'])*100
    del df['S+DM'], df['S-DM']

    # ADX
    df['DX'] = (np.abs(df['+DMI'] - df['-DMI'])/(df['+DMI'] + df['-DMI']))*100
    df['ADX'+str(period)] = df['DX'].ewm(alpha=alpha, adjust=False).mean()
    del df['DX'], df['ATR'], df['TR'], df['-DX'], df['+DX'], df['+DMI'], df['-DMI']

    return df



Days15 = 15
Days50 =50


def add_extras(df):
    df = EVM(df, Days15)
    df = EVM(df, Days50)  
    df = EWMA(df, Days15)
    df = EWMA(df, Days50) 
    df = ROC(df, Days15)
    df = ROC(df, Days50) 
    df = ADX(df, Days15)
    df = ADX(df, Days50)
    return df

df = add_extras(df)

In [ ]:
df = df.dropna()

# Baseline

In [ ]:
df.Date = pd.to_datetime(df.Date)

In [ ]:
def date_to_classic(df):
    dt = df.Date
    df['Year'] = dt.dt.year
    df['Month'] = dt.dt.month
    df['Week'] = dt.dt.week
    df['Days'] = dt.dt.day
    df['quarter'] = dt.dt.quarter
    df.drop("Date", axis=1, inplace=True)


date_to_classic(df)

# Let`s see XGBRegressor Power

In [ ]:
def modelling(X, Y):
    model = xgb.XGBRegressor(
        n_estimators=500,
        max_depth=11,
        learning_rate=0.05,
        subsample=0.9,
        random_state=17,
        tree_method='gpu_hist')
    model.fit(X, Y)
    return model

In [ ]:
MODELS_DANGER = {}

In [ ]:
a=0
for i in df.SecuritiesCode.unique():
    train = df.loc[df.SecuritiesCode == i]
    X = train
    Y = X.pop("Target")
    MODELS_DANGER[i] = modelling(X,Y)
    a+=1
    if a ==100:
        break

# General Model

In [ ]:
general_X = df.copy()
general_Y = general_X.pop("Target")
general_model = modelling(general_X, general_Y)

In [ ]:
env = jpx_tokyo_market_prediction.make_env()

In [ ]:
iter_test = env.iter_test()

In [ ]:
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    ds=[prices, options, financials, trades, secondary_prices, sample_prediction]
    df = sample_prediction[["Date","SecuritiesCode"]]
    prices.drop(["RowId", "ExpectedDividend", "SupervisionFlag","AdjustmentFactor"], axis=1, inplace=True)
    secondary_prices.drop(["RowId", "ExpectedDividend", "SupervisionFlag","AdjustmentFactor"], axis=1, inplace=True)
    main = pd.merge(prices, secondary_prices, how="outer", sort=True)
    main = add_extras(main)
    main = main.dropna()
    df["High"] = main["High"]
    df["Open"] = main["Open"]
    df["Close"] = main["Close"]
    df["Low"] = main["Low"]
    df['Volume'] = main['Volume']
    for i in ['EVM_15','EVM_50', 'EWMA_15', 'EWMA_50', 'Rate_of_Change_15','Rate_of_Change_50', 'ADX15', 'ADX50']:
        df[i] = main[i]
        
    df.Date = pd.to_datetime(df.Date)
    date_to_classic(df)
    PREDICTIONS = []
    for _,v in df.iterrows():
        if v['SecuritiesCode'] in MODELS_DANGER.keys():
            PREDICTIONS.append(float(MODELS_DANGER[v['SecuritiesCode']].predict(pd.DataFrame([v]))))
        else:
            PREDICTIONS.append(float(general_model.predict(pd.DataFrame([v]))))
    print(PREDICTIONS)
    sample_prediction["Prediction"] = PREDICTIONS
    sample_prediction = sample_prediction.sort_values(by = "Prediction", ascending=False)
    sample_prediction.Rank = np.arange(0,2000)
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    sample_prediction.drop(["Prediction"],axis=1)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    env.predict(submission)